In [1]:
import pandas as pd # Data processing, CSV file I/O 
import numpy as np # Linear Algebra
from sklearn.model_selection import train_test_split
from keras.models import Sequential
# The various layers for the Neural Network Model.
# Dense - A layer that is fully connected (densely-connected.)
# Conv2D - A 2-dimensional convolutional layer.
# Dropout - A layer that helps prevent overfitting.
# Flatten - A layer that flattens the input.
# MaxPooling2D - A layer that performs Max Pooling of the Convolutions
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
# Since I have a GPU & I've GPU enabled, I am going to use the GPU version of keras 
# (NOTE: Ignore if you do not have GPU enabled)
import tensorflow as tf
from keras import backend as K
#sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
#K.tensorflow_backend._get_available_gpus()

modelLoaded = False

Using TensorFlow backend.


In [2]:
#dataset = pd.read_csv(filepath_or_buffer="data.csv", 
#                      sep=",")

#trainSet, testSet = train_test_split(dataset,test_size = 0.2)

#trainSet.to_pickle("train_set.pkl")
#testSet.to_pickle("test_set.pkl")

#print(dataset)

In [3]:
trainSet = pd.read_pickle("train_set.pkl")
testSet = pd.read_pickle("test_set.pkl")

In [10]:
model = tf.keras.models.load_model("Models\\CNN_01.model")
modelLoaded = True

In [5]:
###Divisione ed elaborazione del dataset

In [4]:
trainSet_x = trainSet.drop("Label",axis = 1).div(255.0)
trainSet_label = trainSet["Label"]

testSet_x = testSet.drop("Label",axis = 1).div(255.0)
testSet_label = testSet["Label"]

Labels = trainSet_label.unique()
i=0
intToLabel = {}
labelToInt = {}
for l in Labels:
    intToLabel[i] = str(l)
    labelToInt[l] = i
    i+=1
    
trainSet_x = trainSet_x.values.reshape(trainSet.shape[0],28,28,1)
testSet_x = testSet_x.values.reshape(testSet.shape[0],28,28,1)

trainSet_y = []
for l in trainSet_label:
    trainSet_y.append(labelToInt[l])

testSet_y = []
for l in testSet_label:
    testSet_y.append(labelToInt[l])
    
print(len(Labels))

84


In [5]:
#print(Labels)

In [6]:
###

In [7]:
###Creazione e training del modello

In [12]:
modelLoaded = False
INPUT_SHAPE = trainSet_x[0].shape
BATCH_SIZE = 128
EPOCHS = 2
TOT_LABELS = max(trainSet_y)+1

In [17]:
if not modelLoaded:
    model = Sequential()
    
    model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same', input_shape=INPUT_SHAPE))
    model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same', input_shape=INPUT_SHAPE))
    model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'))
    
    #model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Flatten())

    model.add(Dense(512, activation='relu'))
    model.add(Dense(TOT_LABELS, activation='softmax'))
    
    print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 14, 14, 64)        18496     
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 14, 14, 64)        36928     
_________________________________________________________________
flatten_4 (Flatten)          (None, 12544)             0         
_________________________________________________________________
dense_8 (Dense)              (None, 512)               6423040   
__________

model.summary()

In [18]:
if not modelLoaded:
    model.compile(optimizer="adam",
                 loss="sparse_categorical_crossentropy",
                 metrics=["accuracy"])
    model.fit(x=trainSet_x,
              y=trainSet_y,
              epochs=EPOCHS,
              batch_size=BATCH_SIZE,
              validation_split=0.1)

Train on 270701 samples, validate on 30078 samples
Epoch 1/2
270701/270701 [==============================] - 1678s 6ms/step - loss: 0.3019 - acc: 0.9164 - val_loss: 0.1357 - val_acc: 0.9574
Epoch 2/2
270701/270701 [==============================] - 1964s 7ms/step - loss: 0.0983 - acc: 0.9688 - val_loss: 0.0807 - val_acc: 0.9744


In [19]:
if not modelLoaded:
    model.save("CNN.model")

In [ ]:
###

In [ ]:
###Test

In [ ]:
print(testSet_x)
print(testSet_y)

In [20]:
value = model.evaluate(testSet_x, testSet_y)

print(model.metrics_names)
print(value)

75195/75195 [==============================] - 174s 2ms/step
['loss', 'acc']
[0.08272785814927966, 0.973562071946273]


In [11]:
predictions = model.predict(testSet_x)

In [15]:
correct = 0
i=0
for y in testSet_label:
    p = np.argmax(predictions[i])
    #print(str(y) + " - " + intToLabel[p])
    if intToLabel[p] == str(y):
        correct += 1
    i+=1
 
print((correct/len(predictions))*100)

95.53028791807967
